#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [2]:
embeddings=OllamaEmbeddings()
# FAISS: Facebook AI Similarity Search 
# create vector store from documents and embeddings.
db=FAISS.from_documents(docs,embeddings)
db

/var/folders/02/t37nr9yx499fkml2w90sj5x40000gn/T/ipykernel_15046/216211220.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings()


In [3]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [4]:
#### As a Retriever
# We can also convert the vectorstore into a Retriever class. 
# This allows us to easily use it in other LangChain methods, which largely work with retrievers
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [5]:
# Get documents along with their similarity scores, less is more similar.
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='69b3132c-69af-4010-942f-3d7d273396c8', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15594.715)),
 (Document(id='5ec2388a-0be2-4583-abdc-7845e156404b', metadata={'source': 'spe

In [6]:
embedding_vector=embeddings.embed_query(query)
print(embedding_vector)
# Similarity seach by vector
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[1.88955819606781, -3.0957257747650146, 1.968241810798645, 1.4706099033355713, -0.88889479637146, 0.6568133234977722, 1.5207712650299072, -0.4901653826236725, 0.9174358248710632, -2.01017427444458, 1.3012017011642456, -1.9612314701080322, -0.6221703886985779, 1.5269618034362793, -0.49825477600097656, -2.0464158058166504, -0.4237518310546875, 0.06445834040641785, 0.7282382249832153, -1.8396774530410767, -0.6957399249076843, -1.035286545753479, 2.1680562496185303, -2.016876697540283, 0.7547914981842041, -0.9739757776260376, 0.3392753005027771, -1.677051305770874, 0.16935478150844574, -0.8540796637535095, 2.0830483436584473, -2.852626085281372, -2.6640567779541016, 3.773219585418701, 2.2576558589935303, -5.105576515197754, -1.0419189929962158, 1.3776991367340088, -1.258774757385254, -1.382975459098816, -0.46671968698501587, -2.0432188510894775, 2.143169641494751, 0.34756535291671753, 0.9233852028846741, 0.5517531037330627, -0.2569301426410675, 0.7443403005599976, 0.7751494646072388, -1.92

[Document(id='69b3132c-69af-4010-942f-3d7d273396c8', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='5ec2388a-0be2-4583-abdc-7845e156404b', metadata={'source': 'speech.txt'}, page_content='To 

In [7]:
### Saving And Loading
db.save_local("faiss_index")

In [8]:
# allow_dangerous_deserialization=True: I trust this FAISS file — it’s safe to load.

# whether to allow deserialization of the data which involves loading a pickle file. 
# Pickle files can be modified by malicious actors to deliver a malicious payload 
# that results in execution of arbitrary code on your machine.
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)
docs

[Document(id='69b3132c-69af-4010-942f-3d7d273396c8', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='5ec2388a-0be2-4583-abdc-7845e156404b', metadata={'source': 'speech.txt'}, page_content='To 